In [1]:
import os
import pickle
import numpy as np
import math
import cooler
import pandas as pd

In [3]:
pickle_files_directory_c2_m1 = '/commons/groups/gursoy_lab/wlounsberyscaife/ML_poreC/output/01_12/tensors_c2_m1'
cool_directory_c2_m1 = '/commons/groups/gursoy_lab/wlounsberyscaife/ML_poreC/scripts/python_scripts/factor_analysis/cool/data/poreC_cools_c2_m1'

pickle_files_directory_c2_m2 = '/commons/groups/gursoy_lab/wlounsberyscaife/ML_poreC/output/01_12/tensors_c2_m2'
cool_directory_c2_m2 = '/commons/groups/gursoy_lab/wlounsberyscaife/ML_poreC/scripts/python_scripts/factor_analysis/cool/data/poreC_cools_c2_m2'

pickle_files_directory_c2_m3 = '/commons/groups/gursoy_lab/wlounsberyscaife/ML_poreC/output/01_12/tensors_c2_m3'
cool_directory_c2_m3 = '/commons/groups/gursoy_lab/wlounsberyscaife/ML_poreC/scripts/python_scripts/factor_analysis/cool/data/poreC_cools_c2_m3'

pickle_files_directory_c2_test = '/commons/groups/gursoy_lab/wlounsberyscaife/ML_poreC/output/01_12/tensors_c2_test'
cool_directory_c2_test = '/commons/groups/gursoy_lab/wlounsberyscaife/ML_poreC/scripts/python_scripts/factor_analysis/cool/data/poreC_cools_c2_test'

# Base directory for the input and output files
input_dir_cools_c2_m1 = "/commons/groups/gursoy_lab/wlounsberyscaife/ML_poreC/scripts/python_scripts/factor_analysis/cool/data/poreC_cools_c2_m1/"
output_dir_cools_c2_m1 = "/commons/groups/gursoy_lab/wlounsberyscaife/ML_poreC/scripts/python_scripts/factor_analysis/cool/data/poreC_cools_c2_m1_transformed/"

input_dir_cools_c2_m2 = "/commons/groups/gursoy_lab/wlounsberyscaife/ML_poreC/scripts/python_scripts/factor_analysis/cool/data/poreC_cools_c2_m2/"
output_dir_cools_c2_m2 = "/commons/groups/gursoy_lab/wlounsberyscaife/ML_poreC/scripts/python_scripts/factor_analysis/cool/data/poreC_cools_c2_m2_transformed/"

input_dir_cools_c2_m3 = "/commons/groups/gursoy_lab/wlounsberyscaife/ML_poreC/scripts/python_scripts/factor_analysis/cool/data/poreC_cools_c2_m3/"
output_dir_cools_c2_m3 = "/commons/groups/gursoy_lab/wlounsberyscaife/ML_poreC/scripts/python_scripts/factor_analysis/cool/data/poreC_cools_c2_m3_transformed/"

input_dir_cools_c2_test = "/commons/groups/gursoy_lab/wlounsberyscaife/ML_poreC/scripts/python_scripts/factor_analysis/cool/data/poreC_cools_c2_test/"
output_dir_cools_c2_test = "/commons/groups/gursoy_lab/wlounsberyscaife/ML_poreC/scripts/python_scripts/factor_analysis/cool/data/poreC_cools_c2_test_transformed/"

In [4]:
# Define the bin size used in your 2D tensors
bin_size = 100000

def get_chromosome_lengths():
    chromosome_lengths = {}
    chromosome_lengths["chr1"] = 248956422
    chromosome_lengths["chr2"] = 242193529
    chromosome_lengths["chr3"] = 198295559
    chromosome_lengths["chr4"] = 190214555
    chromosome_lengths["chr5"] = 181538259
    chromosome_lengths["chr6"] = 170805979
    chromosome_lengths["chr7"] = 159345973
    chromosome_lengths["chr8"] = 145138636
    chromosome_lengths["chr9"] = 138394717
    chromosome_lengths["chr10"] = 133797422
    chromosome_lengths["chr11"] = 135086622
    chromosome_lengths["chr12"] = 133275309
    chromosome_lengths["chr13"] = 114364328
    chromosome_lengths["chr14"] = 107043718
    chromosome_lengths["chr15"] = 101991189
    chromosome_lengths["chr16"] = 90338345
    chromosome_lengths["chr17"] = 83257441
    chromosome_lengths["chr18"] = 80373285
    chromosome_lengths["chr19"] = 58617616
    chromosome_lengths["chr20"] = 64444167
    chromosome_lengths["chr21"] = 46709983
    chromosome_lengths["chr22"] = 50818468
    return chromosome_lengths

# chromosomes = [str(i) for i in range(1, 23)]
# chromosomes = ["5", "15", "17", "20", "21"]
chromosomes = ["20"]
chromosome_lengths = get_chromosome_lengths()


In [5]:
def load_tensor_from_pickle(my_file_path):
    with open(my_file_path, 'rb') as file:
        tensor = pickle.load(file)
    return tensor.numpy()  # Convert PyTorch tensor to a NumPy array

def get_number_of_bins(chromosome_length, bin_size):
    return math.ceil(chromosome_length / bin_size) + 1

In [6]:
def create_cool_file(interaction_matrix, chromosome, cool_output_path, bin_size):
    # Generate a minimal BED-like dataframe for bins
    n_bins = interaction_matrix.shape[0]
    bins_data = {
        'chrom': [chromosome] * n_bins,
        'start': [i * bin_size for i in range(n_bins)],
        'end': [(i + 1) * bin_size for i in range(n_bins)],
    }
    bins_df = pd.DataFrame(bins_data)

    # Generate a dataframe for the pixel values in the matrix
    pixel_data = {
        'bin1_id': [],
        'bin2_id': [],
        'count': []
    }
    for i in range(n_bins):
        for j in range(i, n_bins):
            count = interaction_matrix[i, j]
            if count > 0:  # We only record non-zero interactions
                pixel_data['bin1_id'].append(i)
                pixel_data['bin2_id'].append(j)
                pixel_data['count'].append(count)
    pixels_df = pd.DataFrame(pixel_data)

    # Create the .cool file (unbalanced)
    cooler.create_cooler(cool_uri=cool_output_path, bins=bins_df, pixels=pixels_df)
    print("Created unbalanced .cool file:", cool_output_path)


In [7]:
def create_and_balance_cool_file(interaction_matrix, chromosome, cool_output_path, bin_size):
    # Generate a minimal BED-like dataframe for bins
    n_bins = interaction_matrix.shape[0]
    bins_data = {
        'chrom': [chromosome] * n_bins,
        'start': [i * bin_size for i in range(n_bins)],
        'end': [(i + 1) * bin_size for i in range(n_bins)],
    }
    bins_df = pd.DataFrame(bins_data)

    # Generate a dataframe for the pixel values in the matrix
    pixel_data = {
        'bin1_id': [],
        'bin2_id': [],
        'count': []
    }
    for i in range(n_bins):
        for j in range(i, n_bins):
            count = interaction_matrix[i, j]
            if count > 0:  # We only record non-zero interactions
                pixel_data['bin1_id'].append(i)
                pixel_data['bin2_id'].append(j)
                pixel_data['count'].append(count)
    pixels_df = pd.DataFrame(pixel_data)

    # Create the .cool file
    cooler.create_cooler(cool_uri=cool_output_path, bins=bins_df, pixels=pixels_df)
    print("Created .cool file:", cool_output_path)

    # Load the created .cool file for balancing
    clr = cooler.Cooler(cool_output_path)

    # Perform balancing
    balanced_clr = cooler.balance_cooler(
        clr, 
        cis_only=True, 
        ignore_diags=2, 
        mad_max=5, 
        min_nnz=10, 
        min_count=0, 
        store=True, 
        store_name='weight'
    )

    print("Balanced .cool file created and stored:", cool_output_path)


In [25]:
for file_name in os.listdir(pickle_files_directory_c2_m2):
    if file_name.endswith('.pickle'):
        chromosome = file_name.split('_')[2]
        if chromosome == 'chr20':
            file_path = os.path.join(pickle_files_directory_c2_m2, file_name)
            interaction_tensor = load_tensor_from_pickle(file_path)
            num_bins = get_number_of_bins(chromosome_lengths[chromosome], bin_size)

            if interaction_tensor.shape == (num_bins, num_bins):
                cool_file_name = file_name.replace('.pickle', '.cool')
                balanced_cool_file_name = cool_file_name.replace('.cool', '_balanced.cool')
                cool_file_path = os.path.join(cool_directory_c2_m2, cool_file_name)
                balanced_cool_file_path = os.path.join(cool_directory_c2_m2, balanced_cool_file_name)
                # create_cool_file(interaction_tensor, chromosome, cool_file_path, bin_size)
                create_and_balance_cool_file(interaction_tensor, chromosome, balanced_cool_file_path, bin_size)
                print("success")


Created .cool file: /commons/groups/gursoy_lab/wlounsberyscaife/ML_poreC/scripts/python_scripts/factor_analysis/cool/data/poreC_cools_c2_m2/NlaIII_GM12878_chr20_tensor_balanced.cool
Balanced .cool file created and stored: /commons/groups/gursoy_lab/wlounsberyscaife/ML_poreC/scripts/python_scripts/factor_analysis/cool/data/poreC_cools_c2_m2/NlaIII_GM12878_chr20_tensor_balanced.cool
success


'\nfor file_name in os.listdir(pickle_files_directory_c2_m1):\n    if file_name.endswith(\'.pickle\'):\n        chromosome = file_name.split(\'_\')[2]\n        if chromosome in chromosome_lengths:\n            file_path = os.path.join(pickle_files_directory_c2_m1, file_name)\n            interaction_tensor = load_tensor_from_pickle(file_path)\n            num_bins = get_number_of_bins(chromosome_lengths[chromosome], bin_size) \n\n            if interaction_tensor.shape == (num_bins, num_bins):\n                cool_file_name = file_name.replace(\'.pickle\', \'.cool\')\n                cool_file_path = os.path.join(cool_directory_c2_m1, cool_file_name)\n                create_cool_file(interaction_tensor, chromosome, cool_file_path, bin_size)\n        else:\n            print(f"Chromosome {chromosome} not found in chromosome lengths dictionary.")\n\n\nprint("\n")\nfor file_name in os.listdir(pickle_files_directory_c2_m3):\n    if file_name.endswith(\'.pickle\'):\n        chromosome = fi

In [8]:
for file_name in os.listdir(pickle_files_directory_c2_m1):
    if file_name.endswith('.pickle'):
        chromosome = file_name.split('_')[2]
        if chromosome == 'chr20':
            file_path = os.path.join(pickle_files_directory_c2_m1, file_name)
            interaction_tensor = load_tensor_from_pickle(file_path)
            num_bins = get_number_of_bins(chromosome_lengths[chromosome], bin_size)

            if interaction_tensor.shape == (num_bins, num_bins):
                cool_file_name = file_name.replace('.pickle', '.cool')
                balanced_cool_file_name = cool_file_name.replace('.cool', '_balanced.cool')
                cool_file_path = os.path.join(cool_directory_c2_m1, cool_file_name)
                balanced_cool_file_path = os.path.join(cool_directory_c2_m1, balanced_cool_file_name)
                # create_cool_file(interaction_tensor, chromosome, cool_file_path, bin_size)
                create_and_balance_cool_file(interaction_tensor, chromosome, balanced_cool_file_path, bin_size)
                print("success")


Created .cool file: /commons/groups/gursoy_lab/wlounsberyscaife/ML_poreC/scripts/python_scripts/factor_analysis/cool/data/poreC_cools_c2_m1/NlaIII_GM12878_chr20_tensor_balanced.cool
Balanced .cool file created and stored: /commons/groups/gursoy_lab/wlounsberyscaife/ML_poreC/scripts/python_scripts/factor_analysis/cool/data/poreC_cools_c2_m1/NlaIII_GM12878_chr20_tensor_balanced.cool
success
